# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fca60f1ca60>
├── 'a' --> tensor([[-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077]])
└── 'x' --> <FastTreeValue 0x7fca60f1cb20>
    └── 'c' --> tensor([[ 1.0628,  1.0622,  0.2754, -1.3692],
                        [-0.9351,  1.0969,  0.3160,  0.1298],
                        [-1.0289, -0.0498,  0.5864,  0.3890]])

In [4]:
t.a

tensor([[-0.2233,  0.0827, -0.6963],
        [-0.8301, -0.5389,  0.6077]])

In [5]:
%timeit t.a

73.9 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fca60f1ca60>
├── 'a' --> tensor([[-0.7525, -0.0193,  0.1042],
│                   [-0.4943,  0.4845, -0.9254]])
└── 'x' --> <FastTreeValue 0x7fca60f1cb20>
    └── 'c' --> tensor([[ 1.0628,  1.0622,  0.2754, -1.3692],
                        [-0.9351,  1.0969,  0.3160,  0.1298],
                        [-1.0289, -0.0498,  0.5864,  0.3890]])

In [7]:
%timeit t.a = new_value

82.1 ns ± 0.139 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077]]),
    x: Batch(
           c: tensor([[ 1.0628,  1.0622,  0.2754, -1.3692],
                      [-0.9351,  1.0969,  0.3160,  0.1298],
                      [-1.0289, -0.0498,  0.5864,  0.3890]]),
       ),
)

In [10]:
b.a

tensor([[-0.2233,  0.0827, -0.6963],
        [-0.8301, -0.5389,  0.6077]])

In [11]:
%timeit b.a

66.1 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5840, -1.4278,  0.7745],
               [ 0.6606, -1.8461, -0.0737]]),
    x: Batch(
           c: tensor([[ 1.0628,  1.0622,  0.2754, -1.3692],
                      [-0.9351,  1.0969,  0.3160,  0.1298],
                      [-1.0289, -0.0498,  0.5864,  0.3890]]),
       ),
)

In [13]:
%timeit b.a = new_value

547 ns ± 6.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

930 ns ± 1.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 26.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

149 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

152 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc982fd85e0>
├── 'a' --> tensor([[[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]],
│           
│                   [[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]],
│           
│                   [[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]],
│           
│                   [[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]],
│           
│                   [[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]],
│           
│                   [[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]],
│           
│                   [[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]],
│           
│                   [[-0.2233,  0.0827, -0.6963],
│                    [-0.8301, -0.5389,  0.6077]]])
└── 'x' --> <FastTreeValue 0x7fc982fd8a60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.6 µs ± 48.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc98304f820>
├── 'a' --> tensor([[-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077],
│                   [-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077],
│                   [-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077],
│                   [-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077],
│                   [-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077],
│                   [-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077],
│                   [-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077],
│                   [-0.2233,  0.0827, -0.6963],
│                   [-0.8301, -0.5389,  0.6077]])
└── 'x' --> <FastTreeValue 0x7fca61f87e50>
    └── 'c' --> tensor([[ 1.0628,  1.0622,  0.2754, -1.3692],
                        [-0.9351,  1.0969,  0.3160,  0.1298],
                 

In [23]:
%timeit t_cat(trees)

31.5 µs ± 93.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.6 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]],
       
               [[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]],
       
               [[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]],
       
               [[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]],
       
               [[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]],
       
               [[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]],
       
               [[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]],
       
               [[-0.2233,  0.0827, -0.6963],
                [-0.8301, -0.5389,  0.6077]]]),
    x: Batch(
           c: tensor([[[ 1.0628,  1.0622,  0.2754, -1.3692],
                       [-0.9351,  1.0969,  0.3160,  0.1298],
                       [-1.0289, -0.0498,  0.5864,  0.3890]],
         

In [26]:
%timeit Batch.stack(batches)

84.3 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077],
               [-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077],
               [-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077],
               [-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077],
               [-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077],
               [-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077],
               [-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077],
               [-0.2233,  0.0827, -0.6963],
               [-0.8301, -0.5389,  0.6077]]),
    x: Batch(
           c: tensor([[ 1.0628,  1.0622,  0.2754, -1.3692],
                      [-0.9351,  1.0969,  0.3160,  0.1298],
                      [-1.0289, -0.0498,  0.5864,  0.3890],
                      [ 1.0628,  1.0622,  0.2754, -1.3692],
                      [-0.9351,  

In [28]:
%timeit Batch.cat(batches)

153 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

374 µs ± 4.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
